# 第7章: データベース
Key Value Store (KVS) やNoSQLによるデータベースの構築・検索を修得します．また，CGIを用いたデモ・システムを開発します．

- LevelDB, MongoDB, JSON, インデックス, 整列, CGI, テンプレートエンジン

[artist.json.gz](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/artist.json.gz)は，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．

```
フィールド	型	内容	例
id	ユニーク識別子	整数	20660
gid	グローバル識別子	文字列	"ecf9f3a3-35e9-4c58-acaa-e707fba45060"
name	アーティスト名	文字列	"Oasis"
sort_name	アーティスト名（辞書順整列用）	文字列	"Oasis"
area	活動場所	文字列	"United Kingdom"
aliases	別名	辞書オブジェクトのリスト	
aliases[].name	別名	文字列	"オアシス"
aliases[].sort_name	別名（整列用）	文字列	"オアシス"
begin	活動開始日	辞書	
begin.year	活動開始年	整数	1991
begin.month	活動開始月	整数	
begin.date	活動開始日	整数	
end	活動終了日	辞書	
end.year	活動終了年	整数	2009
end.month	活動終了月	整数	8
end.date	活動終了日	整数	28
tags	タグ	辞書オブジェクトのリスト	
tags[].count	タグ付けされた回数	整数	1
tags[].value	タグ内容	文字列	"rock"
rating	レーティング	辞書オブジェクト	
rating.count	レーティングの投票数	整数	13
rating.value	レーティングの値（平均値）	整数	86
```
artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

In [3]:
!curl -O http://www.cl.ecei.tohoku.ac.jp/nlp100/data/artist.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 46.9M  100 46.9M    0     0  7412k      0  0:00:06  0:00:06 --:--:-- 7999k 36.2M    0     0  6999k      0  0:00:06  0:00:05  0:00:01 7374k


In [4]:
!gunzip artist.json.gz

In [6]:
!head artist.json

{"name": "WIK▲N", "tags": [{"count": 1, "value": "sillyname"}], "sort_name": "WIK▲N", "ended": true, "gid": "8972b1c1-6482-4750-b51f-596d2edea8b1", "id": 805192}
{"name": "Gustav Ruppke", "sort_name": "Gustav Ruppke", "ended": true, "gid": "b4f76788-7e6f-41b7-ac7b-dfb67f66282e", "type": "Person", "id": 578352}
{"name": "Pete Moutso", "sort_name": "Moutso, Pete", "ended": true, "gid": "49add228-eac5-4de8-836c-d75cde7369c3", "type": "Person", "id": 371203}
{"ended": true, "gid": "c112a400-af49-4665-8bba-741531d962a1", "sort_name": "Zachary", "id": 273232, "name": "Zachary"}
{"name": "The High Level Ranters", "sort_name": "High Level Ranters, The", "ended": true, "gid": "c42eed94-e233-44e2-82b8-3ed6dd9bf318", "type": "Group", "id": 153193}
{"begin": {"year": 1956}, "end": {"year": 1993}, "name": "The Silhouettes", "area": "United States", "sort_name": "Silhouettes, The", "ended": true, "gid": "ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b", "type": "Group", "id": 101060, "aliases": [{"name": "Silh

## 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [7]:
import json
import redis

In [11]:
!redis-server &

OSError: Background processes not supported.

In [55]:
import subprocess
p = subprocess.Popen(['redis-server', '--port', '7777'])

In [56]:
db = redis.Redis(port=7777)

In [62]:
db.set('testkey', 'testval')

True

In [64]:
db.get('testkey')

b'testval'

In [65]:
db.get('nonsense')

In [67]:
db.delete('testkey')

1

In [68]:
db.get('testkey')

In [72]:
with open('artist.json') as f:
    for line in f:
        artist = json.loads(line)
        db.set(artist['name'], artist.get('area', ''))

## 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [78]:
db.get("Al Street")

b'United States'

## 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [79]:
sum(db.get(key) == b'Japan' for key in db.scan_iter())

21946

## 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [80]:
db.flushall()

True

In [82]:
db.keys()

[]

In [84]:
db.lpush('test', 'v2')

1

In [85]:
db.lpush('test', 'v1')

2

In [86]:
db.llen('test')

2

In [87]:
db.lindex('test', 0)

b'v1'

In [88]:
db.lindex('test', 1)

b'v2'

In [89]:
db.lindex('test', 9999)

In [90]:
db.delete('test')

1

In [91]:
db.keys()

[]

In [93]:
with open('artist.json') as f:
    for line in f:
        artist = json.loads(line)
        for tag in artist.get('tags', []):
            db.lpush(artist['name'], f"{tag['value']}:{tag['count']}")

In [96]:
name = 'Infester'
for i in range(db.llen(name)):
    print(db.lindex(name, i))

b'american:1'
b'death metal:1'


In [97]:
db.shutdown()

In [98]:
p.poll()

0

## 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [156]:
p = subprocess.Popen(['mongod', '--dbpath', 'db'])
p.poll()

In [109]:
import pymongo

In [110]:
cli = pymongo.MongoClient()

In [117]:
db = cli.nlp100_64

In [122]:
with open('artist.json') as f:
    db.artist.insert_many(
        json.loads(line)
        for line in f.readlines()
    )

In [123]:
print(db.artist.find_one())

{'_id': ObjectId('5c959924d099ad6af85054cf'), 'name': 'WIK▲N', 'tags': [{'count': 1, 'value': 'sillyname'}], 'sort_name': 'WIK▲N', 'ended': True, 'gid': '8972b1c1-6482-4750-b51f-596d2edea8b1', 'id': 805192}


In [145]:
db.artist.create_index([('name', pymongo.ASCENDING)])

'name_1'

In [146]:
db.artist.create_index([('aliases.name', pymongo.ASCENDING)])

'aliases.name_1'

In [147]:
db.artist.create_index([('tags.value', pymongo.ASCENDING)])

'tags.value_1'

In [148]:
db.artist.create_index([('ratings.value', pymongo.ASCENDING)])

'ratings.value_1'

## 65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．


```
mongo
> show dbs
admin      0.000GB
config     0.000GB
local      0.000GB
nlp100_64  0.135GB
> use nlp100_64
switched to db nlp100_64
> show collections
artist
> db.artist.findOne({"name": "Queen"})
{
	"_id" : ObjectId("5c959932d099ad6af8598bf3"),
	"name" : "Queen",
	"area" : "Japan",
	"gender" : "Female",
	"tags" : [
		{
			"count" : 1,
			"value" : "kamen rider w"
		},
		{
			"count" : 1,
			"value" : "related-akb48"
		}
	],
	"sort_name" : "Queen",
	"ended" : true,
	"gid" : "420ca290-76c5-41af-999e-564d7c71f1a7",
	"type" : "Character",
	"id" : 701492,
	"aliases" : [
		{
			"name" : "Queen",
			"sort_name" : "Queen"
		}
	]
}
> exit
bye
```

In [159]:
db.artist.find_one({'name': 'Queen'})

{'_id': ObjectId('5c959932d099ad6af8598bf3'),
 'name': 'Queen',
 'area': 'Japan',
 'gender': 'Female',
 'tags': [{'count': 1, 'value': 'kamen rider w'},
  {'count': 1, 'value': 'related-akb48'}],
 'sort_name': 'Queen',
 'ended': True,
 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
 'type': 'Character',
 'id': 701492,
 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}

## 66. 検索件数の取得
MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．

```
mongo
> use nlp100_64
switched to db nlp100_64
> db.artist.find({"area": "Japan"}).count()
22821
```

In [160]:
db.artist.find({'area': 'Japan'}).count()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


22821

In [162]:
db.artist.count_documents({'area': 'Japan'})

22821

## 67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．

In [172]:
list(db.artist.find({'aliases.name': 'Queen'}))

[{'_id': ObjectId('5c959932d099ad6af8598bf3'),
  'name': 'Queen',
  'area': 'Japan',
  'gender': 'Female',
  'tags': [{'count': 1, 'value': 'kamen rider w'},
   {'count': 1, 'value': 'related-akb48'}],
  'sort_name': 'Queen',
  'ended': True,
  'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
  'type': 'Character',
  'id': 701492,
  'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}]

## 68. ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [173]:
db.artist.find_one({'tags.value': 'dance'})

{'_id': ObjectId('5c959924d099ad6af8508bcb'),
 'rating': {'count': 1, 'value': 80},
 'name': 'Definition of Sound',
 'tags': [{'count': 1, 'value': 'male vocalists'},
  {'count': 1, 'value': 'dance'},
  {'count': 1, 'value': 'hip hop'},
  {'count': 1, 'value': '90s'},
  {'count': 1, 'value': 'rock and indie'},
  {'count': 1, 'value': 'jazz'}],
 'sort_name': 'Definition of Sound',
 'ended': True,
 'gid': 'f015191f-8d20-4fc9-845f-b4bd6f348967',
 'type': 'Group',
 'id': 35488}

In [175]:
import itertools

In [178]:
list(itertools.islice(range(10), 5))

[0, 1, 2, 3, 4]

In [179]:
cur = db.artist.find({'tags.value': 'dance'}, sort=[('rating.count', pymongo.DESCENDING)])
dance_ratingcount_top10 = list(itertools.islice(cur, 10))
dance_ratingcount_top10

[{'_id': ObjectId('5c959935d099ad6af85b65f0'),
  'rating': {'count': 26, 'value': 88},
  'begin': {'date': 16, 'month': 8, 'year': 1958},
  'name': 'Madonna',
  'area': 'United States',
  'gender': 'Female',
  'tags': [{'count': 1, 'value': 'dance-pop'},
   {'count': 1, 'value': 'electropop'},
   {'count': 1, 'value': 'tell me'},
   {'count': 1, 'value': 'pop and chart'},
   {'count': 1, 'value': 'multiple ipi'},
   {'count': 1, 'value': 'electronic'},
   {'count': 1, 'value': 'américain'},
   {'count': 1, 'value': 'usa'},
   {'count': 1, 'value': 'singer'},
   {'count': 1, 'value': 'chanteur'},
   {'count': 1, 'value': 'american'},
   {'count': 4, 'value': 'pop'},
   {'count': 1, 'value': 'greatest hits'},
   {'count': 1, 'value': 'dance'}],
  'sort_name': 'Madonna',
  'ended': True,
  'gid': '79239441-bfd5-4981-a70c-55c3f15c1287',
  'type': 'Person',
  'id': 89},
 {'_id': ObjectId('5c959934d099ad6af85a8f7a'),
  'rating': {'count': 23, 'value': 84},
  'begin': {'date': 21, 'month': 11

In [180]:
[x['name'] for x in dance_ratingcount_top10]

['Madonna',
 'Björk',
 'The Prodigy',
 'Rihanna',
 'Britney Spears',
 'Maroon 5',
 'Adam Lambert',
 'Fatboy Slim',
 'Basement Jaxx',
 'Cornershop']

## 69. Webアプリケーションの作成
ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．

別のレポジトリ ([nlp100_69](https://github.com/mtannaan/nlp100_69)) にて作成。